<a href="https://colab.research.google.com/github/Nniikkoollaass/forecasting-customer-outflow/blob/model-development/model_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Вирішив зосередитися на логістичній регресії тому, що це ефективний метод для бінарної класифікації -
 прогнозування відтоку клієнтів. Також вибрив підхід аналізу підходу для цієї задачі,
 бо інша людина займається аналізом та підготовкою даних, і я маю час для певної підготовки коду та
  й для вибору підходу для побудови моделі. Використоавши цей час, та зробивши нижче наведені висновки,
   я тому і вибрав для такої задачі і такого Датасету саме логістичну регресію. Та також враховуючи, обмеження в часі, після аналізу підходів, я зупиняюся на цьому.

Тобто модель повинна класифікувати кожного клієнта в одну з двох груп: клієнт, який припинить користуватися послугами компанії
 (покине нас - Клас 1 (позитивний клас)), або клієнт, який продовжить користуватися послугами компанії (залишиться з нами - Клас 0 (негативний клас)).

Детальніше про розробку моделі:
- модель розробляємо, щоб виявити ймовірність відтоку клієнтів на основі зібраних реальних даних
- реалізовувати логіку будемо через підхід логістичної регресії
- аналізуватися будуть всі дані про клієнтів, які наведені в Датасеті
- використовуватимемо мітку - значення (1 або 0), тобто факт відтоку (клієнт може нас покинути) або його відсутність (клієнт певно лишиться з нами).

Я вибрав саме підхід логістичної регресії, бо цей підхід надає ймовірність приналежності до класу для кожної характеристики клієнта
 (надає розуміння впливу будь-якої ознаки на ймовірність відтоку), його досить легко прописати і коригувати (тобто реалізувати),
 добре працює з різними наборами даних та може надати хорошу точність якщо дані правильно підготовлені (зазвичай надає високу точність прогнозування на багатьох наборах даних).

Тобто логістична регресія полягає в її здатності надавати легко інтерпретовані ймовірнісні оцінки,
що робить її особливо корисною для задач бінарної класифікації,
таких як прогнозування відтоку клієнтів.
Це дозволяє аналітикам не тільки передбачати результат, але й розуміти ступінь впевненості у цих прогнозах, що є критично важливим для прийняття бізнес-рішень.

Під час навчання моделі буду використовувати крос-валідацію для налаштування параметрів моделі, для наступного:
- уникнення перенавчання (це коли модель занадто добре підлаштовується під навчальні дані і не може обробляти нові з очікуваними результатами)
- оптимізації гіперпараметрів (Для моделі може існувати багато гіперпараметрів, які потрібно налаштовувати.
  Крос-валідація дозволяє ефективно тестувати різні комбінації гіперпараметрів на декількох підмножинах даних і вибирати оптимальні значення, які забезпечують найкращу продуктивність моделі)
- забезпечення більної стабільності моделі (Крос-валідація використовує кілька підмножин даних для навчання і тестування.
Це дозволяє моделі краще узагальнюватися і бути менш залежною від конкретного поділу даних.)
- підтримання балансу між складністю мобелі та її здатністю оптимально опрацьовувати нові набори даних
- оцінювання моделі через призму різних метрик
Тому крос-валідацію краще застосовувати при налаштуванні параметрів моделі для забезпечення більшої точності,
стабільності та узагальненості (так би мовити "здатності очікувано давати прогноз на реальних, не навчальних даних") моделі,
яка може ефективно працювати на нових даних і допомагає уникнути перенавчання.

Наступні підходи не викорисуються, бо
- 1 - Лінійна регресія - краще підходить для задач передбачення кількісних значень, тоді як завдання прогнозування відтоку клієнтів є бінарною класифікацією
може передбачити значення, що лежать поза межами [0, 1], що не має сенсу для ймовірностей
- 2 - Метод опорних векторів (SVM) - є обчислювально витратними, особливо для великих наборів даних
та не надає безпосередньої інтерпретації у вигляді ймовірностей, що може бути корисним для бізнес-аналітики
- 3 - Дерева рішень - схильні до перенавчання; Хоча дерева рішень легко інтерпретуються, вони можуть бути нестабільними,
оскільки навіть невеликі зміни у даних можуть призвести до значних змін у структурі дерева
- 4 - Кластеризація (K-means) - підходить для задач сегментації або кластеризації, де потрібно розбити дані на групи, але не для бінарної класифікації
- 5 - Нейронні мережі - Навчання нейронних мереж вимагає значних обчислювальних ресурсів і часу, що може бути надмірним для задачі з невеликою кількістю ознак

При оціюванні точноті моделі планую використати метрики точності, повноти, точності передбачення та F1-міри.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# завантаження даних - потім переробиться на підготовлені дані
data = pd.read_csv('/mnt/data/internet_service_churn.csv')

# Розділяємо дані на ознаки (X) та мітки (y)
# видаляємо колонку 'сhurn' з набору даних data, залишаючи всі інші колонки як ознаки. axis=1 означає, що ми видаляємо колонку, а не рядок
x = data.drop('churn', axis=1)  # будуть ознаки - є даними, на основі яких ми будемо робимо прогнози
# вибираємо колонку 'сhurn' як цільову змінну, яку модель буде прогнозувати
y = data['churn'] # це мітка - є тим, що ми намагаємося передбачити

# розподіл даних на навчальну та тестову вибірки
# dикористовуємо train_test_split для розподілу даних на навчальну та тестову вибірки
# test_size: відсоток даних, що будуть використані для тестування
# random_state є параметром, який використовується для фіксації початкового значення генератора випадкових чисел
# фіксоване значення random_state дозволяє будь-кому отримувати однакові результати при кожному запуску коду
x_trained, x_tested, y_trained, y_tested = train_test_split(X, y, test_size=0.2, random_state=123)

# ініціалізація моделі логістичної регресії
# max_iter=1000 - аргумент, який задає максимальну кількість ітерацій алгоритму оптимізації
# параметр max_iter гарантує, що алгоритм не зациклиться і завершиться після певної кількості кроків навіть, якщо не досягне оптимального розв'язку
our_model = LogisticRegression(max_iter=1000)

# параметри для крос-валідації
# визначаємо набір гіперпараметрів, які будуть використовуватися для налаштування моделі
# 'C' - це параметр регуляризації у логістичній регресії. Він контролює ступінь регуляризації (тобто, обмеження складності моделі)
# дозволяє знайти оптимальний баланс між підгонкою та узагальненням
# значення 0.01, 0.1: більш сильна регуляризація (запобігає перенавчанню, але може недонавчити)
# роблять регуляризацію сильнішою, зменшуючи ризик перенавчання, але можуть призвести до недонавчання
# значення 1: стандартне значення, часто використовується як відправна точка
# значення 10, 100: менш сильна регуляризація (може краще підійти для складніших моделей)
# послаблюють регуляризацію, що може допомогти моделі краще адаптуватися до тренувальних даних, але підвищує ризик перенавчання
# 'solver' - це алгоритм, який використовується для оптимізації функції втрат у логістичній регресії
# значення lbfgs': Limited-memory Broyden–Fletcher–Goldfarb–Shanno (ефективний для невеликих і середніх за розміром наборів даних)
# значення liblinear': Л=лінійний солвер, підходить для великих наборів даних
# всі ці параметри важливі для забезпечення оптимальної продуктивності моделі, запобігання перенавчанню та адаптації до структури даних.
parameters = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}

# крос-валідація
# використовуємо GridSearchCV для пошуку кращих параметрів
# налаштовуємо пошук найкращих гіперпараметрів для моделі логістичної регресії за допомогою методу крос-валідації
# 'GridSearchCV' - це клас у бібліотеці scikit-learn, який використовується для автоматичного підбору найкращих гіперпараметрів моделі
# він перебирає всі можливі комбінації гіперпараметрів і оцінює продуктивність кожної комбінації за допомогою крос-валідації
# вибирає ту комбінацію гіперпараметрів, яка показала найкращу середню точність на крос-валідації
# 'estimator' вказує на модель, для якої ми хочемо підібрати гіперпараметри
# 'param_grid' задає сітку гіперпараметрів, які потрібно перебирати
# це словник, де ключі - це імена параметрів, а значення - списки можливих значень цих параметрів
# 'cv' означає кількість фолдів у крос-валідації
# ми використовуємо 5-кратну крос-валідацію, тобто дані розбиваються на 5 частин,
# і кожна частина по черзі використовується як тестовий набір, тоді як інші частини використовуються як тренувальні набори
# 'scoring' задає метрику, яку використовують для оцінки продуктивності моделі
# таким чином ми забезпечуємо автоматичний пошук оптимальних гіперпараметрів,
# запобігаємо перенавчанню та також використання крос-валідації з різними фолдами
# дає більш об'єктивну оцінку продуктивності моделі порівняно з простою тренувальною та тестовою вибіркою
search = GridSearchCV(estimator=our_model, param_grid=parameters, cv=5, scoring='accuracy')

# навчаємо модель з кращими параметрами на повному навчальному наборі даних
# метод fit тренує об'єкт GridSearchCV на заданих навчальних даних
search.fit(x_trained, y_trained)

# отримуємо найкращі гіперпараметри, знайдені у процесі пошуку за допомогою GridSearchCV
# '.best_params_' - це атрибут об'єкта GridSearchCV, який зберігає найкращі гіперпараметри, знайдені під час пошуку
# та містить словник з найкращими значеннями гіперпараметрів
# це нам дозволяє:
# - оптимізувати продуктивність - використання найкращих гіперпараметрів дозволяє досягти максимальної точності моделі на основі навчальних даних
# - автоматизувати процес - GridSearchCV автоматично перебирає всі можливі комбінації гіперпараметрів, що значно спрощує роботу
# - отримати надійність оцінки - використання крос-валідації забезпечує надійність результатів, оскільки модель оцінюється на різних підмножинах даних
best_parameters = search.best_params_
# друкуємо найкращі параметри
print(f'Best parameters: {best_parameters}')

# навчання моделі з кращими параметрами
best_model = search.best_estimator_
best_model.fit(x_trained, y_trained)

# робимо передбачення на тестовій вибірці
y_predicted = best_model.predict(x_tested)

# оцінка моделі
accuracy = accuracy_score(y_tested, y_predicted)
recall = recall_score(y_tested, y_predicted)
precision = precision_score(y_tested, y_predicted)
f1 = f1_score(y_tested, y_predicted)

print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'Precision: {precision}')
print(f'F1 Score: {f1}')
